In [1]:
# # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !pip install transformers
# !pip install transformers scipy ftfy accelerate

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [2]:
from recomendaciones_comsint import Recomendador

c:\Users\jsaur\anaconda3\envs\ENV1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
recomendador128 = Recomendador(fuente='recetario_mexicano_small.csv')
recomendador128.CargarModelo(emb_size=128, version=4)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Modelo Modelos/Modelo_Nut_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.
Modelo Modelos/Modelo_Precios_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.


In [4]:
recomendador128.FiltrarRecetario_por_CanastaBasica(similitud=0.6, verbose=False)
df_platillos_recomendados2 = recomendador128.Calcular_InfoNutricional(verbose=False)

Buscando recetas con ingredientes de la canasta básica... 



100%|██████████| 15/15 [00:00<00:00, 33.20it/s]


 

 4 platillos encontrados con similitud mayor a 0.6
1/1 [==============================] - 0s 204ms/step


In [5]:
df2 = df_platillos_recomendados2.sort_values(by=['similitud'], ascending=False)
df2

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
0,Receta de Aporreadillo rojo,500 gramos de Cesina\n 500 gramos de Jitomate\...,0.6,133.75,7.14,14.73,4.82,0.3
1,Receta de Discada norteña,00 gramos de Carne de res\n 500 gramos de Pulp...,0.6,8627.15,433.08,741.20,442.31,0.7
2,Guiso de filete a la mexicana,3 unidades de Jitomates maduros picados en cub...,0.6,9780.98,460.53,991.73,427.63,0.3
3,Receta de Pozole mexicano estilo Nayarit,1 kilogramo de Maíz\n 1 cucharada sopera de Ca...,0.6,341.26,16.05,34.94,12.04,0.3


In [48]:
df3 = df2[df2['puntaje_platillo']>=0.3]
df3

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
0,Receta de Aporreadillo rojo,500 gramos de Cesina\n 500 gramos de Jitomate\...,0.6,133.75,7.14,14.73,4.82,0.3
1,Receta de Discada norteña,00 gramos de Carne de res\n 500 gramos de Pulp...,0.6,8627.15,433.08,741.20,442.31,0.7
2,Guiso de filete a la mexicana,3 unidades de Jitomates maduros picados en cub...,0.6,9780.98,460.53,991.73,427.63,0.3
3,Receta de Pozole mexicano estilo Nayarit,1 kilogramo de Maíz\n 1 cucharada sopera de Ca...,0.6,341.26,16.05,34.94,12.04,0.3


In [49]:
df4 = recomendador128.Calcular_Precios(df3, verbose=False)
df4

1/1 [==============================] - 0s 30ms/step


,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
0,Receta de Aporreadillo rojo,500 gramos de Cesina\n 500 gramos de Jitomate\...,0.6,133.75,7.14,14.73,4.82,0.3,133.71
1,Receta de Discada norteña,00 gramos de Carne de res\n 500 gramos de Pulp...,0.6,8627.15,433.08,741.20,442.31,0.7,63.62
2,Guiso de filete a la mexicana,3 unidades de Jitomates maduros picados en cub...,0.6,9780.98,460.53,991.73,427.63,0.3,391.27
3,Receta de Pozole mexicano estilo Nayarit,1 kilogramo de Maíz\n 1 cucharada sopera de Ca...,0.6,341.26,16.05,34.94,12.04,0.3,38.49


In [53]:
df4[df4['costo_receta']>=50].sort_values(by=['costo_receta','similitud','kcal'], ascending=True)

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
1,Receta de Discada norteña,00 gramos de Carne de res\n 500 gramos de Pulp...,0.6,8627.15,433.08,741.20,442.31,0.7,63.62
0,Receta de Aporreadillo rojo,500 gramos de Cesina\n 500 gramos de Jitomate\...,0.6,133.75,7.14,14.73,4.82,0.3,133.71
2,Guiso de filete a la mexicana,3 unidades de Jitomates maduros picados en cub...,0.6,9780.98,460.53,991.73,427.63,0.3,391.27


In [39]:
resultados = df4.to_dict(orient='records')

In [44]:
resultados[0]

{'nombre_del_platillo': 'Guiso de filete a la mexicana',
 'ingredientes': '3 unidades de Jitomates maduros picados en cubos\n 5 unidades de Tomates verdes picados en cubos\n 1 unidad de Pimiento morrón verde picado en cubos\n 1 unidad de Chile serrano verde picado\n ½ unidad de Cebolla picada en cubos\n ¼ unidad de Cebolla\n 1 diente de Ajo picado\n ½ manojo de Cilantro pequeño\n 1 pizca de Sal o consomé de pollo en polvo\n 1 pizca de Pimienta molida\n 5 filetes de Milanesa o filetes de res picada en cuadros pequeños',
 'similitud': 0.6,
 'kcal': 9780.98,
 'proteinas_gr': 460.53,
 'carbohidratos_gr': 991.73,
 'grasas_gr': 427.63,
 'puntaje_platillo': 0.3,
 'costo_receta': 391.27}